In [43]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime
import uuid

In [60]:
child_age = str(6)
child_name = "Caitlin"
child_interests = "Barbie"
child_name_no_spaces = child_name.replace(" ", "_")
today = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
folder_name = os.path.join('app', 'storybuilder', 'stories', f'{child_name_no_spaces}_{today}_{uuid.uuid4().hex}')
print(folder_name)

app/storybuilder/stories/Caitlin_2024-12-15_21-16-56_56f191f66a854aaabb7067e5f37fe42a


In [61]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

#load the generator prompt
with open('app/storybuilder/prompts/generator.md', 'r') as file:
    md_generator_prompt = file.read()

with open('app/storybuilder/prompts/childs_info.txt', 'r') as file:
    child_info = file.read()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": md_generator_prompt},
              {"role": "user", "content": f"""{child_info}""".format(child_age=child_age,child_name=child_name, child_interests=child_interests)}],
    stream=False,
)

print(completion.choices[0].message.content)


# main.md
@story: Barbie's Magical Adventure
@flow: [
    "fairy_land_intro",
    "meeting_barbie",
    "finding_the_dress"
]
@state: {
    global: {
        friends_made: [],
        magical_items: []
    }
}

# characters.md
@character: barbie
prompt: A stylish doll with long blonde hair, sparkling blue eyes, and a lovely pink dress.
personality: {
    trait: "charming and adventurous",
    goal: "to help Caitlin discover magical treasures",
    speech_style: "enthusiastic and kind"
}

# scenes.md
@scene: fairy_land
id: fairy_land_1
prompt: A colorful fairy land with twinkling stars and bright flowers, sparkling all around.
mood: joyful
time: midday

@scene: meeting_barbie
id: meeting_barbie_2
prompt: A cozy clearing with a beautiful rainbow and glittering butterflies fluttering above.
mood: magical
time: midday

@scene: finding_the_dress
id: finding_the_dress_3
prompt: A shimmering cave filled with sparkling gemstones and a soft, glowing light.
mood: exciting
time: midday

# scene_f

In [62]:
#split the completion into multiple files 
# each file starts with #scene_name.md

def split_scenes(completion_text):
    # Split the text into lines
    lines = completion_text.split('\n')
    
    current_scene = None
    current_content = []
    scenes = {}
    
    for line in lines:
        # Check for new scene marker
        if line.startswith('#') and line.endswith('.md'):
            # If we have a current scene, save it
            if current_scene:
                scenes[current_scene] = '\n'.join(current_content)
            
            # Start new scene
            current_scene = line[1:].strip()  # Remove # and .md
            current_content = []
        elif current_scene:
            current_content.append(line)
    
    # Save the last scene
    if current_scene and current_content:
        scenes[current_scene] = '\n'.join(current_content)
    
    return scenes




In [63]:
# Get the scenes from completion
scenes = split_scenes(completion.choices[0].message.content)


In [64]:
#make stories directory if not exists

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Write each scene to a file
for scene_name, content in scenes.items():
    with open(os.path.join(folder_name, scene_name), 'w') as f:
        f.write(content.strip())
    print(f'Created scene file: {scene_name}')

Created scene file: main.md
Created scene file: characters.md
Created scene file: scenes.md
Created scene file: scene_fairy_land_intro.md
Created scene file: scene_meeting_barbie.md
Created scene file: scene_finding_the_dress.md
